## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import numpy as np                                                              
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import scipy
import tensorflow as tf                                                         
import data.data_selector as ds                                                   
import analysis.analysis_picker as ap
import utils.plot_functions as pf                                               

## Model parameters

In [ ]:
class analysis_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_vh"
    self.version = "2.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

# Computed params
analysis_params = analysis_params()
analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
analysis_params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+analysis_params.model_name)

## Load analyzer

In [ ]:
analyzer = ap.get_analyzer(analysis_params.model_type)
analyzer.setup(analysis_params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=analysis_params.save_info)
analyzer.model_name = analysis_params.model_name

In [ ]:
def random_angles(init, num_steps, step_size, momentum_weight=0.0):
  init = np.asarray(init)
  prev_step = 0.0
  angles = [init]
  for step in range(1, num_steps):
    delta = np.random.normal(0.0, 1.0, size=init.shape)
    delta_angle = momentum_weight * prev_step + step_size * delta
    new_angle = angles[step-1] + delta_angle
    for dim in range(new_angle.ndim-1):
      if new_angle[dim] > np.pi:
        new_angle[dim] = new_angle[dim] - np.pi
    if new_angle[-1] > 2*np.pi:
      new_angle[-1] = new_angle[-1] - np.pi
    angles.append(new_angle)
  return np.stack(angles, axis=0)

In [ ]:
num_steps = 500
step_size = np.pi/10
momentum_weight = 2.0
init = [0, 0]
angles = random_angles(init, num_steps, step_size, momentum_weight)
points = np.zeros((num_steps, 3))
for angle_id, angle in enumerate(angles):
  points[angle_id, 0] = np.cos(angle[0])
  points[angle_id, 1] = np.sin(angle[0])*np.cos(angle[1])
  points[angle_id, 2] = np.sin(angle[0])*np.sin(angle[1])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
line, = ax.plot(points[:2,0], points[:2,1], zs=points[:2,2], color='k', linewidth=0.5, alpha=0.8)

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])

def init():
  line.set_xdata([np.nan])
  line.set_ydata([np.nan])
  line.set_3d_properties([np.nan])
  return line,

def animate(i):
  line.set_xdata(points[:i, 0])
  line.set_ydata(points[:i, 1])
  line.set_3d_properties(points[:i, 2])
  return line,

ani = animation.FuncAnimation(fig, animate, frames=num_steps, init_func=init, interval=25, blit=True)
ani.save(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_test_plot.mp4")

ax.scatter(points[:,0], points[:,1], points[:,2], color='r', s=1.0)
plt.show()

In [ ]:
# https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates
import itertools

target_group = 1
num_examples_per_dimension = 10
num_steps = 1000
step_size = np.pi/10
momentum_weight = 2.0

num_groups = analyzer.model_params.num_groups
num_neurons = analyzer.model_params.num_neurons
num_neurons_per_group = num_neurons // num_groups

# Traversal of the space one axis at a time
#angles = [np.linspace(0, np.pi, num_examples_per_dimension) for _ in range(num_neurons_per_group-2)]
#angles += [np.linspace(0, 2*np.pi, num_examples_per_dimension)]
#angles = [angle for angle in itertools.product(*angles)]

# Traversal of the space via random walk
init = [0]*num_neurons_per_group
angles = random_angles(init, num_steps, step_size, momentum_weight)

In [ ]:
zs = []
for angle in angles:
  z = np.zeros((num_groups, num_neurons_per_group))
  for group_neuron_idx in range(num_neurons_per_group):
    if group_neuron_idx == 0:
      z[target_group, group_neuron_idx] = np.cos(angle[group_neuron_idx])
    elif group_neuron_idx > 0 and group_neuron_idx <= num_neurons_per_group-2:
      prev_group_angles = [np.sin(angle[prev_group_neuron_idx])
        for prev_group_neuron_idx in range(group_neuron_idx)]
      z[target_group, group_neuron_idx] = np.prod(prev_group_angles)*np.cos(angle[group_neuron_idx])
    else: # group_neuron_idx == num_neurons_per_group - 1
      prev_group_angles = [np.sin(angle[prev_group_neuron_idx])
        for prev_group_neuron_idx in range(group_neuron_idx)]
      z[target_group, group_neuron_idx] = np.prod(prev_group_angles)
  zs.append(z)
zs = np.stack(zs, axis=0)

sigmas = np.zeros((zs.shape[0], analyzer.model_params.num_groups))
sigmas[:, target_group] = 1

In [ ]:
input_shape = [zs.shape[0]]+ analyzer.model.get_input_shape()[1:]
feed_dict = analyzer.model.get_feed_dict(np.zeros(input_shape), is_test=True)
feed_dict[analyzer.sigmas] = sigmas
feed_dict[analyzer.zs] = zs
recons = analyzer.evaluate_tf_tensor(analyzer.group_recons, feed_dict)

In [ ]:
fig = plt.figure()
ax = pf.clear_axis(fig.add_subplot())
ims = []
for i in range(recons.shape[0]):
  recon = recons[i].reshape([analyzer.model_params.patch_edge_size]*2)
  im = ax.imshow(recon, animated=True, cmap="Greys_r")
  ims.append([im])
ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group_"+str(target_group)+"_recons.mp4")

### Full image recons

In [ ]:
normed_image = (
  (analyzer.full_image - np.min(analyzer.full_image))
  / (np.max(analyzer.full_image) - np.min(analyzer.full_image))).astype(np.float32)

normed_recon = (
  (analyzer.full_recon - np.min(analyzer.full_recon))
  / (np.max(analyzer.full_recon) - np.min(analyzer.full_recon))).astype(np.float32)

fig, ax = plt.subplots(1, 2, figsize=(12,12))
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(np.squeeze(normed_image), cmap="Greys_r")
ax[0].set_title("Input Image", fontsize=16)
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(np.squeeze(normed_recon), cmap="Greys_r")
percent_active = "{:.2f}".format(analyzer.recon_frac_act*100)
psnr = "{:.2f}".format(compare_psnr(normed_image, normed_recon, data_range=1))
ax[1].set_title("Reconstruction\n"+percent_active+" percent active"+"\n"+"PSNR = "+psnr, fontsize=16)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_image_recon.png", transparent=True,
  bbox_inches="tight")

### New Analysis

### MNIST

### VanHateren

## Construct analysis plots

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats)
fig.savefig(analyzer.analysis_out_dir+"vis/location_frequency_centers.png")

In [ ]:
neuron_angles = analyzer.get_neuron_angles(analyzer.bf_stats)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(neuron_angles[1], num_bins=50, angle_min=0, angle_max=180, figsize=(8,8))
angle_hist_fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_angle_histogram.png")

In [ ]:
weight_indices = np.stack([np.array(id_list) for id_list in analyzer.model.module.group_ids], axis=0)
pooling_weights = np.stack([analyzer.evals["lca_subspace/weights/w:0"][:, id_list] for id_list in analyzer.model.module.group_ids], axis=1)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits import axes_grid1
import utils.data_processing as dp
def plot_group_weights(weights, group_ids, title="", figsize=None,  save_filename=None):
  """
    weights: [np.ndarray] of shape [num_neurons, num_input_y, num_input_x]
    group_ids: [list of lists] containing ids for each group [[,]*neurons_per_group,]*num_groups
  """
  num_neurons = weights.shape[0]
  for weight_id in range(num_neurons):
    weights[weight_id,...] = weights[weight_id,...] - weights[weight_id,...].mean()
    weights[weight_id,...] = weights[weight_id,...] / (weights[weight_id,...].max()-weights[weight_id,...].min())
  vmin = np.min(weights)
  vmax = np.max(weights)
  indices = [idx for id_list in group_ids for idx in id_list]
  num_groups = len(group_ids)
  num_groups_x = int(np.floor(np.sqrt(num_groups)))
  num_groups_y = int(np.ceil(np.sqrt(num_groups)))
  num_neurons_per_group = len(group_ids[0])
  num_neurons_x = int(np.floor(np.sqrt(num_neurons_per_group)))
  num_neurons_y = int(np.ceil(np.sqrt(num_neurons_per_group)))
  outer_spacing = 0.20
  inner_spacing = 0.1
  fig = plt.figure(figsize=figsize)
  gs1 = gridspec.GridSpec(num_groups_y, num_groups_x,
    hspace=outer_spacing*num_groups_y/(num_groups_x+num_groups_y),
    wspace=outer_spacing*num_groups_x/(num_groups_x+num_groups_y))
  neuron_index = 0
  for group_plot_id in np.ndindex((num_groups_y, num_groups_x)):
    gs_inner = gridspec.GridSpecFromSubplotSpec(num_neurons_y, num_neurons_x, gs1[group_plot_id],
      hspace=inner_spacing*num_neurons_y/(num_neurons_x+num_neurons_y),
      wspace=inner_spacing*num_neurons_x/(num_neurons_x+num_neurons_y))
    for inner_plot_id in np.ndindex((num_neurons_y, num_neurons_x)):
      ax = pf.clear_axis(fig.add_subplot(gs_inner[inner_plot_id]))
      ax.set_aspect("equal")
      if neuron_index < num_neurons:
        ax.imshow(weights[indices[neuron_index], ...], cmap="Greys_r", vmin=vmin, vmax=vmax)
        neuron_index += 1
  fig.suptitle(title, y=0.9, x=0.5, fontsize=20)
  if save_filename is not None:
    fig.savefig(save_filename)
    plt.close(fig)
    return None
  plt.show()
  return fig

In [ ]:
weights = np.reshape(analyzer.evals["lca_subspace/weights/w:0"].T, [analyzer.model.params.num_neurons,
      int(np.sqrt(analyzer.model.params.num_pixels)), int(np.sqrt(analyzer.model.params.num_pixels))])
weight_fig = plot_group_weights(np.squeeze(weights), analyzer.model.module.group_ids,
  title="Dictionary", figsize=(18,18))
weight_fig.savefig(analyzer.analysis_out_dir+"/vis/"+"group_phi.png")